In [5]:
# Imports
import networkx as nx
import numpy as np
import cvxpy as cp

In [6]:
# Creating the test graph
G = nx.MultiDiGraph()
G.add_edge(1, 2, weight=1)
G.add_edge(1, 3, weight=5)
G.add_edge(2, 4, weight=1)
G.add_edge(3, 4, weight=5)

#list(G.edges(data=True))
desiredPath = [1, 3, 4]

In [7]:
#Shortest Path

def shortestPath(graph, source, target):

    inf = 1000000
        
    n = len(graph.nodes())

    m = len(graph.edges())

    weights = []

    edges = []
    edgeIndex = {}
    for (i, j, w) in graph.edges(data='weight'):
        # Edges Index
        edgeIndex[i,j] = len(edges)
        # Edge Weight
        weights.append(w)
        # Add edge
        edges.append([i, j])
        
        
    # Add inverse edges with infinity weight
    for (i, j) in graph.edges():
        if (j, i) not in graph.edges():
            # Edges Index
            edgeIndex[j,i] = len(edges)
            # Edge Weight
            weights.append(inf)
            # Add edge
            edges.append([j, i])
            
            graph.add_edge(j, i, weight=inf)
        
        
    nodeIndex = {}
    nodes = []
    for n in graph.nodes:
        # Node Index
        nodeIndex[n] = len(nodes)
        # Add Node
        nodes.append(n)


    # LP:

    # Variables
    x = cp.Variable(len(edges), boolean=True)

    # Weight
    w = np.asarray(weights)

    # Constraints
    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))

    for i in range(len(nodes)):
        for neighbour in graph.adj[nodes[i]]:
            # Filling A
            j = edgeIndex[nodes[i], neighbour]
            A[i,j] = 1
            j = edgeIndex[neighbour, nodes[i]]
            A[i,j] =-1
            
        # Filling b
        if nodes[i] == source:
            b[i] = 1
        if nodes[i] == target:
            b[i] = -1

                
    # Cost function
    cost = w.T @ x

    #Create constraints.
    constraints = [A @ x == b]

    # Form objective.
    obj = cp.Minimize(cost)

    # Form problem.
    prob = cp.Problem(obj, constraints)

    # Solve the problem
    prob.solve(verbose=True)#Detailed
    #prob.solve(solver='ECOS_BB')  # Returns the optimal value.
    print("\nThe optimal value is", prob.value)
    print("A solution x is")
    print(x.value)
    # print("A dual solution is")
    # print(prob.constraints[0].dual_value)
    
    return x.value, w, b, A

In [8]:
x, w, b, A = shortestPath(G, 1, 4)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jun 26 01:17:58 AM: Your problem has 8 variables, 1 constraints, and 0 parameters.
(CVXPY) Jun 26 01:17:58 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 26 01:17:58 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 26 01:17:58 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.


SolverError: 

                    You need a mixed-integer solver for this model. Refer to the documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    for discussion on this topic.

                    Quick fix 1: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer linear programming
                    solver `GLPK`. If your problem is nonlinear then you can install SCIP
                    (pip install pyscipopt).

                    Quick fix 2: you can explicitly specify solver='ECOS_BB'. This may result
                    in incorrect solutions and is not recommended.
                

In [172]:
w

array([      1,       5,       1,       5, 1000000, 1000000, 1000000,
       1000000])

In [ ]:
def optInvMILP(graph, desiredPath, allowedSpeedTypes):
    
    
    #cost_type = "label_changes"   # "weights" or "label_changes"

    # auxiliary variables
    edge2index = {}
    edges = []
    edge2varnodeindex = {}
    varnodes = []
    weights = []
    for (i,j) in graph.edges:
        edge2index[i,j] = len(edges)
        edges.append([i,j])
        # edge2index[j,i] = len(edges)
        # edges.append([j,i])
        # if not graph.nodes[i]["portal"]:
        #     vn = i
        # else:
        #     vn = j
        # if vn in varnodes:
        #     idx = varnodes.index(vn)
        # else:
        #     idx = len(varnodes)
        varnodes.append(vn)
        #edge2varnodeindex[i,j] = idx
        #edge2varnodeindex[j,i] = idx
        weights.append(graph[i][j]["weight"])
        
    node2index = {}
    nodes = []
    for n in graph.nodes:
        node2index[n] = len(nodes)
        nodes.append(n)
        if n == desiredPath[0]:
            s = node2index[n]
        if n == desiredPath[-1]:
            t = node2index[n]
            
            
    weights = np.array(weights)
    
    # l_original
    l_original = np.zeros(len(varnodes) * len(allowedSpeedTypes))
    for (i,j) in graph.edges:
        idx = edge2varnodeindex[i,j]
        node = varnodes[idx]
        for k in range(len(allowedSpeedTypes)):
            if allowedSpeedTypes[k] == graph.nodes[node]["area"]:
                l_original[len(allowedSpeedTypes) * idx + k] = 1
            else:
                l_original[len(allowedSpeedTypes) * idx + k] = 0

    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))
    for i in range(len(nodes)):
        for nei in graph.adj[nodes[i]]:
            j = edge2index[nodes[i], nei]
            A[i,j] = 1
            j = edge2index[nei, nodes[i]]
            A[i,j] =-1
            if i == s:
                b[i] = 1
            if i == t:
                b[i] =-1
            
        
    # optimal x
    path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    xstar = np.zeros(len(edges))
    for p in range(len(path)-1):
        j = edge2index[path[p], path[p+1]]
        xstar[j] = 1

    # desired x
    xzero = np.zeros(len(edges))
    for p in range(len(desiredPath)-1):
        j = edge2index[desiredPath[p], desiredPath[p+1]]
        xzero[j] = 1
    
    return



In [ ]:
# Solving using the function

In [ ]:
#ORIGINAL
# import similaritymeasures
# import matplotlib
# import matplotlib.pyplot as plt
# import time
# import random
# import pdb
# import os
# import rospy
# from dynamic_reconfigure.server import Server
# from recast_explanations_ros.cfg import ExplanationsConfig
# from geometry_msgs.msg import Point
# from visualization_msgs.msg import Marker, MarkerArray
# from std_msgs.msg import ColorRGBA
# from recast_ros.srv import RecastProjectSrv, RecastProjectSrvRequest
# from recast_ros.srv import RecastPathSrv, RecastPathSrvRequest
# from recast_ros.msg import RecastGraph, RecastGraphNode
# from itertools import islice, product
# from tabulate import tabulate
# from queue import PriorityQueue


def optInvMILP2(graph, desiredPath, areaCosts, allowedAreaTypes, exact=True, verbose=True):

  # variables:
  #   x_j:      indicator variable, whether edge j is part of the shortest path
  #   A_ij:     node-arc incidence matrix (rows are nodes, columns are edges) = 1 if j leaves i, -1 if j enters i, 0 otherwise
  #   b_i:      difference between entering and leaving edges = 1 if i start, -1 if i target, 0 otherwise
  #   pi_i:     dual variable
  #   lambda_j: dual variable
  #   c_j:      edge cost = dist_j * ac_0 * l_0 + dist_j * ac_1 * l_1 + ... = sum_(k in areas) dist_j * ac_k * l_ik
  #   l_ik:     node area one-hot encoding

  # problems:
  #   SP:  min  c.x,
  #        s.t. Ax=b, x>=0
  #   ISP: min  |l-l'|,
  #        s.t. sum_i a_ij * pi_i = sum_(k in areas) dist_j * ac_k * l_ik,              for all j in desired path
  #             sum_i a_ij * pi_i + lambda_j = sum_(k in areas) dist_j * ac_k * l_ik,   for all j not in desired path
  #             sum_k l_ik = 1                                                          for all i
  #             lambda >= 0,                                                            for all j not in desired path.

  cost_type = "label_changes"   # "weights" or "label_changes"

  # auxiliary variables
  edge2index = {}
  edges = []
  edge2varnodeindex = {}
  varnodes = []
  weights = []
  for (i,j) in graph.edges:
    edge2index[i,j] = len(edges)
    edges.append([i,j])
    edge2index[j,i] = len(edges)
    edges.append([j,i])
    if not graph.nodes[i]["portal"]:
      vn = i
    else:
      vn = j
    if vn in varnodes:
      idx = varnodes.index(vn)
    else:
      idx = len(varnodes)
      varnodes.append(vn)
    edge2varnodeindex[i,j] = idx
    edge2varnodeindex[j,i] = idx
    weights.append(graph[i][j]["weight"])
  node2index = {}
  nodes = []
  for n in graph.nodes:
    node2index[n] = len(nodes)
    nodes.append(n)
    if n == desiredPath[0]:
      s = node2index[n]
    if n == desiredPath[-1]:
      t = node2index[n]
  weights = np.array(weights)

  # l_original
  l_original = np.zeros(len(varnodes) * len(allowedAreaTypes))
  for (i,j) in graph.edges:
    idx = edge2varnodeindex[i,j]
    node = varnodes[idx]
    for k in range(len(allowedAreaTypes)):
      if allowedAreaTypes[k] == graph.nodes[node]["area"]:
        l_original[len(allowedAreaTypes) * idx + k] = 1
      else:
        l_original[len(allowedAreaTypes) * idx + k] = 0

  # Ax = b
  A = np.zeros([len(nodes), len(edges)])
  b = np.zeros(len(nodes))
  for i in range(len(nodes)):
    for nei in graph.adj[nodes[i]]:
      j = edge2index[nodes[i], nei]
      A[i,j] = 1
      j = edge2index[nei, nodes[i]]
      A[i,j] =-1
    if i == s:
      b[i] = 1
    if i == t:
      b[i] =-1

  # optimal x
  path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
  xstar = np.zeros(len(edges))
  for p in range(len(path)-1):
    j = edge2index[path[p], path[p+1]]
    xstar[j] = 1

  # desired x
  xzero = np.zeros(len(edges))
  for p in range(len(desiredPath)-1):
    j = edge2index[desiredPath[p], desiredPath[p+1]]
    xzero[j] = 1

  # weights for L1norm (not-on-path-penalty)
  w = np.array([1]*len(l_original))
  for i in range(len(varnodes)):
    if varnodes[i] not in desiredPath[1:-1]:
      for k in range(len(allowedAreaTypes)):
        w[len(allowedAreaTypes) * i + k] *= 10

  # inverse optimization problem
  l_ = cp.Variable(len(l_original), boolean=True)
  pi_ = cp.Variable(len(nodes))
  lambda_ = cp.Variable(len(edges))
  # cost
  if cost_type == "weights":
    cost = 0
    j = 0
    for edge in graph.edges:
      i = edge2varnodeindex[edge[0], edge[1]]
      # edge's new cost d_j = sum_(k in areas) dist_j * ac_k * l_ik
      d_j = 0
      dist_j = dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
      for k in range(len(allowedAreaTypes)):
        ac_k = areaCosts[allowedAreaTypes[k]]
        d_j += dist_j * ac_k * l_[len(allowedAreaTypes) * i + k]
      cost += cp.abs(d_j - weights[j])
      j += 1
  else:
    cost = cp.norm1(cp.multiply(l_ - l_original, w))  # cost = cp.norm1(l_ - l_original)
  # constraints
  constraints = []
  for j in range(len(edges)):
    edge = edges[j]
    i = edge2varnodeindex[edge[0], edge[1]]
    # edge's new cost d_j = sum_(k in areas) dist_j * ac_k * l_ik
    d_j = 0
    dist_j = dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
    for k in range(len(allowedAreaTypes)):
      ac_k = areaCosts[allowedAreaTypes[k]]
      d_j += dist_j * ac_k * l_[len(allowedAreaTypes) * i + k]
    if xzero[j] == 1:
      # sum_i a_ij * pi_i = d_j,              for all j in desired path
      constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
    else:
      # sum_i a_ij * pi_i + lambda_j = d_j,   for all j not in desired path
      constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
  # sum_k l_ik = 1, for all i
  for i in range(len(varnodes)):
    idx = len(allowedAreaTypes) * i
    constraints.append( cp.sum(l_[idx:idx+len(allowedAreaTypes)]) == 1 )
  # lambda >= 0, for all j not in desired path.
  for j in range(len(edges)):
    if xzero[j] == 0:
      constraints.append( lambda_[j] >= 0 )

  # solve with cvxpy
  prob = cp.Problem(cp.Minimize(cost), constraints)
  if exact:
    value = prob.solve(solver=cp.MOSEK, mosek_params={"MSK_DPAR_MIO_MAX_TIME":-1, "MSK_DPAR_MIO_TOL_REL_GAP":0, "MSK_IPAR_MIO_CUT_CLIQUE":0, "MSK_IPAR_MIO_CUT_CMIR":0, "MSK_IPAR_MIO_CUT_GMI":0, "MSK_IPAR_MIO_CUT_SELECTION_LEVEL":0, "MSK_IPAR_MIO_FEASPUMP_LEVEL":1, "MSK_IPAR_MIO_VB_DETECTION_LEVEL":1}, verbose=verbose)
  else:
    value = prob.solve(solver=cp.GUROBI, verbose=verbose)
  if value == float('inf'):
    rospy.loginfo("  inverse shortest path MILP failed")
    return []

  # TODO: can solve every 90s, warm starting from previous solution, until optimality or time budget
  #       this way can return multiple solutions of better and better cost

  # new graph
  newGraph = graph.copy()
  changed = 0
  for j in range(len(edges)):
    edge = edges[j]
    i = edge2varnodeindex[edge[0], edge[1]]
    area = -1
    for k in range(len(allowedAreaTypes)):
      if l_.value[len(allowedAreaTypes) * i + k] == 1:
        area = allowedAreaTypes[k]
        break
    if area != graph[edge[0]][edge[1]]["area"]:
      changed += 1
    newGraph[edge[0]][edge[1]]["area"] = area
    newGraph[edge[0]][edge[1]]["cost"] = areaCosts[area]
    newGraph[edge[0]][edge[1]]["weight"] = areaCosts[area] * dist(graph.nodes[edge[0]]["point"], graph.nodes[edge[1]]["point"])
    if not newGraph.nodes[edge[0]]["portal"]:
      newGraph.nodes[edge[0]]["area"] = area
      newGraph.nodes[edge[0]]["cost"] = areaCosts[area]
    if not newGraph.nodes[edge[1]]["portal"]:
      newGraph.nodes[edge[1]]["area"] = area
      newGraph.nodes[edge[1]]["cost"] = areaCosts[area]

  # sanity check
  new_path = nx.shortest_path(newGraph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
  if getCost(graph, new_path) != getCost(graph, desiredPath):
    rospy.logwarn("  new shortest path is not the desired one")
  elif verbose:
    rospy.loginfo("  inverse shortest path: success")

  # changed entries
  if verbose:
    rospy.loginfo("  changed labels: %d" % changed)

  #pdb.set_trace()
  return l_.value, newGraph
